<img src="./assets/uca.png" alt="Tech Logo" align="center" height="800" width="800"/>

<h1 align="left" style="color:#000051;font-size: 30px">TP : Classifiez des émotions vocales avec du deep learning</h1>

Pour ce TP, vous implémenterez un classifieur à base de réseau de neurones sur des données combinant les jeux de données RAVDESS et TESS avec la librairie keras. 
Repartez de ce notebook Jupyter qui passe en revue chaque partie du TP pour présenter des preuves et une analyse de vos résultats

<h1 align="left" style="color:#000051;font-size: 25px">Partie 2 : Développement et évaluation du modèle de deep learning</h1>

Maintenant que les données sont pré-traitées, nous nous attacherons dans cette partie à créer puis évaluer des modèles de deep learning à même d'apprendre à reconnaitre les 4 types d'émotion à partir des données dont nous disposons

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span>💾 <strong>A propos des jeux de données </strong></span></h2>

Pour ce TP, nous utiliserons 2 jeux de données

- **RAVDESS : Ryerson Audio-Visual Database of Emotional Speech and Song**
https://zenodo.org/record/1188976#.X4sE0tDXKUl
  - RAVDESS a été enregistrée avec 24 acteurs professionnels (12 femmes, 12 hommes), prononçant deux phrases lexicalement identiques avec un accent nord-américain neutre. Chaque phrase est prononcée avec deux niveaux d'intensité émotionnelle (normal, fort).
  - **1440 fichiers** = 24 acteurs x 60 fichiers audio par acteur
  - **8 émotions** (neutre, calme, joie, tristesse, colère, peur, dégout, surprise).



- **TESS : Toronto Emotional Speech Set**
https://tspace.library.utoronto.ca/handle/1807/24487
  - Ces données ont été enregistrées par le Northwestern University Auditory. Un ensemble de 200 mots cibles ont été prononcés dans la phrase "Dites le mot _____" par deux actrices (âgées de 26 et 64 ans) et des enregistrements ont été réalisés lorsque ces phrases ont été prononcées avec chacune des sept émotions décrites ci-dessous.
Les deux actrices ont été recrutées dans la région de Toronto. Les deux actrices parlent l'anglais comme première langue, ont fait des études universitaires et ont une formation musicale.
  - **2800 fichiers** = 2 acteurs x 200 phrases x 7 émotions
  - **7 émotions** (neutre, joie, tristesse, colère, peur, dégoût, surprise)('calme' ne fait pas partie de cette BD) 

Pour ce TP, pour des considérations de volume de données, nous n'avons retenu que 4 des émotions:
- **neutre**
- **joie**
- **tristesse**
- **colère**

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span>📥 <strong>1. Import des librairies </strong></span></h2>

Si vous avez besoin d'installer des libraires Python pour ce TP, décommentez et exécutez la cellule ci-dessous

In [ ]:
#!pip3 install numpy==1.18.5
#!pip3 install pydub
#!pip3 install librosa
#!pip3 install noisereduce
#!pip3 install matplotlib
#!pip3 install IPython
#!pip3 install tensorflow
#!pip3 install scikit-learn
#!pip3 install scipy
#!pip3 install pandas

In [ ]:
# Librairies de la bibliothèque standard Python
import os
import random

# Librairies de calcul numérique
import numpy as np
import pandas as pd

# Librairies de traitement audio
from pydub import AudioSegment, effects
import librosa
import soundfile as sf
from scipy.io import wavfile
import noisereduce as nr
from librosa import display   

# Librairies de data-visualization
import matplotlib.pyplot as plt
import IPython.display as ipd 
import seaborn as sns
%matplotlib inline

# Librairies de machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
import time
import tensorflow
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Permute, Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D, Bidirectional, Conv2D
from tensorflow.keras.layers import GRU, CuDNNGRU, LSTM, Bidirectional, BatchNormalization, Reshape, MaxPooling2D, Flatten
from tensorflow.keras.utils import normalize, plot_model
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint


<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span>🔎 <strong>2. Chargement des données d'entrainement et labels</strong></span></h2>

Avant de commencer, chargeons les données d'entrainement et les labels que nous avons précédemment pré-traitées. 

Pour ce faire, utilisons également la librairie pickle.

In [ ]:
import pickle
with open('../1.DataPreprocessing/features.array', 'rb') as features:
    features_array = pickle.load(features)

In [ ]:
import pickle
with open('../1.DataPreprocessing/labels.array', 'rb') as labels:
    labels_array = pickle.load(labels)

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span>🧮 <strong>3. Normalisation des données d'entrainement</strong></span></h2>

En machine learning, la normalisation permet de converger plus rapidement vers le minimum d'erreur et d'éviter la prédominance d'une caractéristique du jeu de données sur une autre.

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Ecrivez le code permettant de normaliser les données d'entrainement <strong>features_array</strong> afin de ramener les valeurs entre 0 et 1</span></p>

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
normalize
 : </span> <a href="https://www.kite.com/python/docs/sklearn.preprocessing.normalize">https://www.kite.com/python/docs/sklearn.preprocessing.normalize</a></p> 

In [ ]:
from sklearn.preprocessing import normalize

features_array = normalize(features_array)

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span>🪓 <strong>4. Partitionnement des données</strong></span></h2>

Le jeu de données  ́etant désormais généré, il est désormais nécessaire de le structurer afin de réaliser l’apprentissage automatique du classifieur. Le jeu de données doit être réparti en :
- Un jeu de données pour l'entrainement
- Un jeu de données pour la validation
- Un jeu de données pour le test

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓ En apprentissage automatique, quel est le rôle du jeu de données d’entraînement, de celui de validation et de celui de test ?</span></p>

Pour ce faire, la librairie scikit-learn dispose de la fonction **train_test_split()** permettant de partitionner les données 

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Ecrivez le code permettant de partitionner les données d'entrainement <strong>features_array</strong> en sous-ensemble d'entrainement et en sous-ensemble de test. Faites attention à la répartition équitable des classes dans les données d'entrainement et de test</span></p>

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
sklearn.model_selection.train_test_split
 : </span> <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html</a></p> 

In [ ]:
from sklearn.model_selection import train_test_split


Utilisez le code ci-dessous pour afficher la répartition des classes dans vos données, avec la supposition que vos labels d'entrainement sont dans la variable **y_train** et que vos labels de test sont dans la variable **y_test**

In [ ]:
import pandas as pd
y_train_series = pd.Series(y_train)
y_train_series.value_counts(normalize=True).plot(kind="pie", shadow=True, autopct="%1.1f", figsize=(12, 10), title="Répartition des classes dans les données d'entrainement")
plt.show()

y_test_series = pd.Series(y_test)
y_test_series.value_counts(normalize=True).plot(kind="pie", shadow=True, autopct="%1.1f", figsize=(12, 10), title="Répartition des classes dans les données d'entrainement")
plt.show()

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span>🧠 <strong>5. Conception de modèles de deep learning</strong></span></h2>

Nous approchons du but, nous allons commencer à concevoir puis entrainer des modèles de deep learning pour modéliser notre problème de reconnaissance d'émotions.

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span>🧠 <strong>5.1 Définition des hyperparamètres </strong></span></h3>

Dans la conception d’un modèle de reseau de neurones, les hyperparamètres permettent de configurer differents  ́elements de l’architecture neuronale, et de superviser comment le modèle apprend

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓ Voici quelques hyperparamètres, savez-vous à quoi elles servent dans la conception et l’apprentissage du modèle ?</span></p>

<img src="./assets/hyperparameters.png" alt="Tech Logo" align="center" height="300" width="300"/>

Voici les valeurs des hyperparamètres à utiliser pour la conception du modèle

In [ ]:
#Model hyperparameters
BATCH_SIZE = 16
INIT_LR = 1e-3
EPOCHS = 50
ACTIVATION_FUNCTION = "relu"
LOSS_TYPE = "sparse_categorical_crossentropy"

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span>🧠 <strong>5.2 Définition des callbacks </strong></span></h3>

Dans Keras, les callbacks permettent de réaliser des traitements périodiques pendant l’apprentissage du modèle, au début ou à la fin de l’apprentissage, au debut ou la fin de chaque itération(epoch), au début ou à la fin de chaque batch. Les callbacks peuvent ainsi permettre d’influer sur ou d’observer l’apprentissage en cours

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓ Pouvez-vous expliquer comment chacun de ces callbacks influe sur le processus d’entraînement ?</span></p>

<img src="./assets/callbacks.png" alt="Tech Logo" align="center" height="300" width="300"/>

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Ecrivez le code permettant de créer 4 instances de callbacks permettant de réaliser des appels de fonctions prédéfinis:  
    <ul style="text-align: left; font-size: 16px; color:#131fcf">
            <li><strong>checkpointer</strong>: doit permettre d'enregistrer à chaque epoch un fichier de poids "model.h5" dans le répertoire CHECKPOINTS_FOLDER en veillant à n'enregistrer le modèle que lorsque la précision (accuracy) s'améliore</li>
            <li><strong>reduce_lr</strong>:doit permettre de diviser par deux le learning rate à chaque fois la précision (accuracy) sur le dataset de validation ne diminue pas pendant 10 epochs, en veillant à ce que le learning ne descende jamais en dessous de 1e-10.</li>
        <li><strong>early_stop</strong>: doit permettre d'arrêter l'entraînement du modèle lorsque la précision (accuracy) sur le dataset de validation n'augmente pas pendant 25 epochs</li>
        <li><strong>tensorboard</strong>: doit permettre d'enregistrer les logs et métadonnées de l'entraînement dans le répertoire CHECKPOINTS_FOLDER</li>

</ul>
    </span></p>

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
Quick Start with Tensorflow Callbacks
 : </span> <a href="https://www.analyticsvidhya.com/blog/2021/08/quick-start-with-tensorflow-callbacks/">https://www.analyticsvidhya.com/blog/2021/08/quick-start-with-tensorflow-callbacks/

In [ ]:
CHECKPOINTS_FOLDER = "./checkpoints"


<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span>🧠 <strong>5.3 Conception de l'architecture neuronale </strong></span></h3>

Nous allons ici coder l'architecture neuronale de notre réseau. Dans Keras, bien souvent, un réseau de neurones est défini comme une séquence de couches connectées les unes aux autres. C'est pour cela que Keras propose un objet modèle appelé **Sequential()**

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Ecrivez le code dans la fonction  <strong>build_architecture</strong> permettant de définir séquentiellement l'architecture du réseau de neurones </span></p>

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  L'architecture neuronale à coder séquentiellement est la suivante :  

<ul style="text-align: left; font-size: 16px; color:#131fcf">
    
<strong><li>Couche convolutionnelle #1</li></strong>
    <ul>
        <li>Nombre de filtres convolutionnels : 32</li>
        <li>Taille des filtres : 15</li>
        <li>Stride : 4</li>
        <li>Fonction d'activation : relu</li>
    </ul>

<strong><li>Couche BatchNormalization</li></strong>

<strong><li>Couche convolutionnelle #2</li></strong>
    <ul>
        <li>Nombre de filtres convolutionnels : 64</li>
        <li>Taille des filtres : 15</li>
        <li>Stride : 4</li>
        <li>Fonction d'activation : relu</li>
    </ul>

<strong><li>Couche BatchNormalization</li></strong>

<strong><li>Couche convolutionnelle #3</li></strong>
    <ul>
        <li>Nombre de filtres convolutionnels : 64</li>
        <li>Taille des filtres : 15</li>
        <li>Stride : 4</li>
        <li>Fonction d'activation : relu</li>
    </ul>

<strong><li>Couche BatchNormalization</li></strong>

<strong><li>Couche reccurente LSTM #1</li></strong>
    <ul>
        <li>Nombre d'unités : 64</li>
    </ul>

<strong><li>Couche dense #1</li></strong>
    <ul>
        <li>Nombre d'unités : 64</li>
        <li>Fonction d'activation : relu</li>
    </ul>
 
<strong><li>Couche dense #2</li></strong>
    <ul>
        <li>Nombre d'unités : ?</li>
        <li>Fonction d'activation : softmax</li>
    </ul>


</span>

</p>

</ul>

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
How Do Convolutional Layers Work in Deep Learning Neural Networks?
 : </span> <a href="https://machinelearningmastery.com/convolutional-layers-for-deep-learning-neural-networks/">https://machinelearningmastery.com/convolutional-layers-for-deep-learning-neural-networks/</a></p> 

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
Conv1D layer : </span> <a href="https://keras.io/api/layers/convolution_layers/convolution1d/">https://keras.io/api/layers/convolution_layers/convolution1d/</a></p> 

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
LSTM layer : </span> <a href="https://keras.io/api/layers/recurrent_layers/lstm/">https://keras.io/api/layers/recurrent_layers/lstm/</a></p> 

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
BatchNormalization layer : </span> <a href="https://keras.io/api/layers/normalization_layers/batch_normalization/">https://keras.io/api/layers/normalization_layers/batch_normalization/</a></p> 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM, Conv1D, BatchNormalization

def build_architecture():
    
    '''
    Fonction permettant de définir l'architecture d'un réseau neuronal

            Parameters:
                    Rien

                    

            Returns:
                    model (Keras Model Sequential) : modèle séquentiel de Keras
    '''
        
    
    return model

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Ecrivez le code permettant d'afficher l'architecture du modèle </span></p>

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
How to Visualize a Deep Learning Neural Network Model in Keras : </span> <a href="https://machinelearningmastery.com/visualize-deep-learning-neural-network-model-keras/">https://machinelearningmastery.com/visualize-deep-learning-neural-network-model-keras/</a></p> 

In [ ]:
model = build_architecture()


<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span>🧠 <strong>5.4 Compilation de l'architecture neuronale </strong></span></h3>

Maintenant que le modèle est défini, nous pouvons le compiler.

La compilation du modèle utilise les bibliothèques numériques efficaces en back-end telles que Theano ou TensorFlow. Le backend choisit automatiquement la meilleure façon de représenter le réseau pour l'entrainement et de faire des prédictions. La compilation découvre également sur quel matériel de votre ordinateur exécuter les différents calculs nécessaires pendant l'entrainement et la prédiction : CPU ou GPU.

Lors de la compilation, nous devons spécifier certaines propriétés supplémentaires requises lors de l'entrainement du réseau. N'oubliez pas qu'entrainer un réseau signifie trouver le meilleur ensemble de poids pour approximer les entrées aux sorties dans notre ensemble de données.

Nous devons spécifier :
- la fonction de perte à utiliser pour évaluer l'erreur, 
- l'optimiseur qui sera utilisé pour les valeurs de poids qui minimisent la fonction de perte 
- toutes les métriques que nous aimerions collecter et monitorer pendant l'entrainement.

<img src="./assets/compile.png" alt="Tech Logo" align="center" height="800" width="800"/>

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Ecrivez le code permettant de compiler l'architecture du réseau de neurones en utilisant une partie des hyperparamètres définis plus haut. Monitorez la métrique accuracy</span></p>

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
How to compile a keras model? : </span> <a href="https://www.projectpro.io/recipes/compile-keras-model">https://www.projectpro.io/recipes/compile-keras-model/</a></p> 

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span>🧠 <strong>5.5 Entrainement du modèle </strong></span></h3>

Nous avons défini notre modèle et l'avons compilé pour un calcul efficace. Il est maintenant temps d'exécuter l'entrainement du modèle sur certaines données.

Nous pouvons entraîner notre modèle sur nos données en appelant la fonction **fit()** sur le modèle.

<img src="./assets/training.png" alt="Tech Logo" align="center" height="800" width="800"/>

L'entrainement se déroule sur des "epoch" et chaque "epoch" est divisée en "batch".

- "epoch" : Itération complète correspondant au traitement de toutes les données du jeu de données d'apprentissage par le modèle.
- "batch" : Lot de plusieurs données soumis séquentiellement au modèle au cours d'une "epoch" avant que les poids ne soient mis à jour.

Une "epoch" est composée d'un ou plusieurs "batch" en fonction de la taille de "batch" choisie. Le modèle est entrainé sur plusieurs "epoch".

- Le processus d'apprentissage s'exécutera donc pendant un nombre fixe d'itérations à travers l'ensemble de données appelé epochs, que nous devons spécifier à l'aide de l'argument epochs. 
- Nous devons également définir la taille des batchs à l'aide de l'argument batch_size.

Nous voulons entraîner suffisamment le modèle pour qu'il apprenne une bonnne (ou assez bonne) approximation des données d'entrées. Le modèle aura toujours une erreur, mais la quantité d'erreur se stabilisera à un certain moment pour une configuration de modèle donnée. **C'est ce qu'on appelle la convergence des modèles. **

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Ecrivez le code permettant d'entrainer le réseau de neurones convolutionnel et recuurent défini et compilé. 
    Lancez l'entrainement pendant 50 itérations, en appliquant la liste de callbacks précédemment définis. Récupérez l'historique de l'entrainement dans la variable training_history

 </span></p>

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
Visualizing training performance with TensorFlow 2 and Keras : </span> <a href="https://www.machinecurve.com/index.php/2019/10/08/how-to-visualize-the-training-process-in-keras/">https://www.machinecurve.com/index.php/2019/10/08/how-to-visualize-the-training-process-in-keras/</a></p> 

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span>📈<strong>6. Visualisation des courbes d'apprentissage</strong></span></h2>

Maintenant que l'entrainement est terminé, il est important de voir comment s'est passé l'apprentissage. Il est possible de visualiser la dynamique de l'entrainement en visualisant les courbes d'apprentissage.

In [ ]:
def plot_results(training_history):
    
        
    '''
    Fonction permettant d'afficher les courbes d'apprentissage du modèle sur les données d'entrainement

            Parameters:
                    training_history (History object): Un objet d'histoire. Son attribut History.history est 
                                                       un enregistrement des valeurs de perte d'entraînement et 
                                                       des valeurs de métriques à des épochs successives, ainsi
                                                       que des valeurs de perte de validation et des valeurs de 
                                                       métriques de validation 
                    

            Returns:
                    Rien
    '''
    
    plt.plot(training_history.history['acc'])
    plt.plot(training_history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    plt.plot(training_history.history['loss'])
    plt.plot(training_history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'val'], loc = 'upper left')
    plt.show()

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Appelez la fonction plot_results avec comme argument l'historique de l'entrainement pour afficher les courbes d'apprentissage </span></p>

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span>👨🏽‍🏫 <strong>7. Evaluation du modèle</strong></span></h2>

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span>👨🏽‍🏫 <strong>7.1 Evaluation empirique sur le jeu de données de test </strong></span></h3>

Nous avons entrainé notre réseau de neurones sur le jeu de données et nous pouvons évaluer les performances du réseau sur le jeu de données de test

Vous pouvez évaluer votre modèle  à l'aide de la fonction **evaluate()**.

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Ecrivez le code permettant d'évaluer les performances du modèle sur les données de test</span></p>

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
How to evaluate a keras model? : </span> <a href="https://www.projectpro.io/recipes/evaluate-keras-model">https://www.projectpro.io/recipes/evaluate-keras-model</a></p> 

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓ Que sont les deux valeurs renvoyées par l'évaluation du modèle ?</span></p>

<h3 style="text-align: left; color:#20a08d; font-size: 20px"><span>👨🏽‍🏫 <strong>7.2 Calcul de la matrice de confusion </strong></span></h3>

In [ ]:
EMOTION_DICT = {0:"neutre", 1:"joie", 2:"tristesse", 3:"colere"}
list(EMOTION_DICT.values())

Allons un peu plus dans le détail de la performance du modèle sur le jeu de données de test. Regardons sur quelle classe le modèle se trompe et avec quelles classes la confusion est la plus grande. Cela peut se faire via une matrice de confusion

<p style="text-align: left; font-size: 16px; color:#131fcf"><span>🖥️  Ecrivez le code dans la fonction  <strong>compute_confusion_matrix</strong> permettant de calculer et d'afficher la matrice de confusion puis appelez cette fonction sur les données de test </span></p>

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
Generate classification report and confusion matrix in Python : </span> <a href="https://www.projectpro.io/recipes/generate-classification-report-and-confusion-matrix-in-python">https://www.projectpro.io/recipes/generate-classification-report-and-confusion-matrix-in-python</a></p> 

<p style="text-align: left; font-size: 16px; color:#ec8f1a"><span>📚  
sklearn.metrics.ConfusionMatrixDisplay : </span> <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html">https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html</a></p> 

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def compute_confusion_matrix(model, X_test, y_test):
    
    '''
    Fonction pour calculer et afficher la matrice de confusion

            Parameters:
                    model (Keras Model): Modèle Keras entrainé sur le jeu d'entrainé 

            Returns:
                    Rien : affiche la matrice de confusion
                    

    '''
        

<p style="text-align: left; font-size: 16px; color:#7a0f43"><span>❓ Pour chaque classe d'émotion, avec quelle émotion la confusion du modèle est la plus grande (en d'autres termes, quelle émotion est la plus représentée dans les erreurs de prédiction pour chaque émotion ?</span></p>

<h2 style="text-align: left; color:#20a08d; font-size: 25px"><span>
📼 <strong>8. Sauvegarde du modèle entrainé</strong></span></h2>

Nous allons ici sauvegarder sur notre disque le modèle que nous avons entrainé en vue d'une utilisation ultérieure.

Pour ce faire, nous utiliserons la fonction save de keras qui permet d'enregistrer des modèles.

In [ ]:
model.save('./model.h5')